In [ ]:
# Project 4 : Distracted Driver MultiAction Classification


import numpy as np
import pandas as pd
import random

# Data: https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=0
# Input data files are kept in the "../Project4/Data/" directory

import os
print(os.listdir("../Project4/Data"))

In [ ]:
!pip install tensorflow
#!pip uninstall tensorflow-gpu==2.10.0 -y

In [ ]:
#import tensorflow as tf
#print(tf.__version__)
#tf.test.is_gpu_available(
#    cuda_only=False,
#    min_cuda_compute_capability=None
#)

In [ ]:
# defining the path and classes.
directory = '../Project4/Data/imgs/train'
test_directory = '../Project4/Data/imgs/test/'
classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']

In [ ]:
# defining a shape to be used for our models.
img_size = 240 #128

In [ ]:
!pip install opencv-python
!pip install opencv-python-headless

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
# Train class image for display.
for i in classes:
    path = os.path.join(directory,i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
        RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        plt.imshow(RGB_img)
        plt.show()
        break
    break

In [ ]:
# Test class image for display.
test_array = []
for img in os.listdir(test_directory):
    img_array = cv2.imread(os.path.join(test_directory,img),cv2.IMREAD_COLOR)
    RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    test_array = RGB_img
    plt.imshow(RGB_img)
    plt.show()
    break

In [ ]:
# checkking image size using shape.
print(img_array.shape)

In [ ]:
# trying out the resize image functionality
new_img = cv2.resize(test_array,(img_size,img_size))
print(new_img.shape)
plt.imshow(new_img)
plt.show()


In [ ]:
# creating a training dataset.
training_data = []
i = 0
def create_training_data():
    for category in classes:
        path = os.path.join(directory,category)
        class_num = classes.index(category)
        
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
            RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
            new_img = cv2.resize(RGB_img,(img_size,img_size))
            training_data.append([new_img,class_num])

In [ ]:
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
random.shuffle(training_data)
x = []
y = []

In [ ]:
for features, label in training_data:
    x.append(features)
    y.append(label)

In [ ]:
len(x),len(y)

In [ ]:
x[0].shape

In [ ]:
len(x)

In [ ]:
x = np.array(x).reshape(-1,img_size,img_size,3)
#x = np.memmap(x).reshape(-1,img_size,img_size,3)
x[0].shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=96)

In [ ]:
del x,y

In [ ]:
import gc
gc.collect()

In [ ]:
#import keras
from tensorflow import keras 

import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
from keras import utils

In [ ]:
Y_train = utils.to_categorical(y_train,num_classes=10)
Y_test = utils.to_categorical(y_test,num_classes=10)

In [ ]:
from tensorflow.keras import datasets, layers, models, callbacks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, BatchNormalization

In [ ]:
model = models.Sequential()
## CNN 1
#model.add(Conv2D(32,(3,3),activation='relu',input_shape=(240,240,3)))
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))
## CNN 2
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))
## CNN 3
model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.5))
## Dense & Output
model.add(Flatten())
model.add(Dense(units = 512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units = 128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
callback = [callbacks.EarlyStopping(monitor='val_accuracy',patience=5)]
batch_size = 70
n_epochs = 15

In [ ]:
results = model.fit(x_train,Y_train,
                    batch_size=batch_size,epochs=n_epochs,
                    verbose=1,
                    validation_data=(x_test,Y_test),
                    callbacks=callback)

In [ ]:
# Plot training & validation accuracy values
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

# Plot training & validation loss values
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

## Predict

In [ ]:
#pred= model.predict_classes(x_test)
predictions = model.predict(x_test)
pred = np.argmax(predictions, axis=1)

print(pred)

## Confusion Matrix

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, pred)
plt.show()

## Classification Report

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, pred))

##  Classify images into these 10 classes - Classes DefinitionsClasses Definitions
    c0: safe driving
    c1: texting - right
    c2: talking on the phone - right
    c3: texting - left
    c4: talking on the phone - left
    c5: operating the radio
    c6: drinking
    c7: reaching behind
    c8: hair and makeup
    c9: talking to passenger

## Testing using random image

In [ ]:
random_filename = random.choice([
    x for x in os.listdir(test_directory)
    if os.path.isfile(os.path.join(test_directory, x))])
random_filename=test_directory+random_filename
print(random_filename)
img_array = cv2.imread(random_filename,cv2.IMREAD_COLOR)
RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
new_img = cv2.resize(RGB_img,(img_size,img_size))
predictions_tst = model.predict(new_img.reshape(-1,img_size,img_size,3))
pred_tst = np.argmax(predictions_tst, axis=1)
print('Prediction : {}'.format(pred_tst))
plt.imshow(new_img)
plt.show()


In [ ]:
#Output file is saved in the Hierarchical Data Format (HDF) version 5
model.save_weights('./prj4_distracted_driver_multiAction_classification_weights.h5', overwrite=True)

In [ ]:
#Output file is saved in the Hierarchical Data Format (HDF) version 5
model.save('./prj4_distracted_driver_multiAction_classification.h5')